# Optimal Driving - Value Function Contours

This notebook produces contour plots of the value function for the Optimal Driving Framework.

## Import Statements:

In [31]:
import numpy as np
import matplotlib.style as style
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
from mpl_toolkits import mplot3d
import matplotlib.ticker as mticker
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import animation
from matplotlib.animation import FuncAnimation, PillowWriter
import math
import os
import sys

### In-text plots toggle:

Uncomment the code in the cell below to view the plots in a separate window.

In [32]:
#%matplotlib qt

## Problem Selection:

Select problem results to view according to numbering scheme:

    1. Stationary (Phase $G$).
    2. Red-Green.
    3. Yellow-Red-Green.
    4. Uncertain Green Duration, 2 light lengths.
    5. Uncertain Green Duration, 3 light lengths.

In [33]:
indexChoices = [1,2,3,4,5]
problemSelector = int(input("Problem Number: ")) #user specified timeslice to plot
if problemSelector not in indexChoices: 
    print ("No such problem index, invalid choice'")
    sys.exit() #kill the program if user-specified N is out of bounds

Problem Number: 1


## Read in Parameters:

In [34]:
paramFileName = 'FileName'

if problemSelector == 1:
    paramFileName = '../output/stationary_params'
elif problemSelector == 2:
    paramFileName = '../output/rg_params'
elif problemSelector == 3:
    paramFileName = '../output/yrg_params'
elif problemSelector == 4:
    paramFileName = '../output/uncertain_green_params_2L'
elif problemSelector == 5:
    paramFileName = '../output/uncertain_green_params_3L'

In [35]:
gInUncertainGreenPhase = 1; 

with open('%s.txt' %paramFileName) as f:    
    
    #Parameter Key
    paramKey = f.readline()
    #paramVals = [float(num) for num in line1.split(' ')]
    
    #Param values
    line2 = f.readline()
    paramVals = [float(num) for num in line2.split(' ')]
    
    line3 =f.readline()
    
    #Uncertain green info
    line4 = f.readline()
    stochLightInfo = [float(num) for num in line4.split(' ')]
    
    line5 = f.readline()
    
    #Sampling info (not currently used)
    line6 = f.readline()
    samplingInfo = [float(num) for num in line6.split(' ')]

### Parameter Definitions:

In [36]:
gDPos = paramVals[0]

gDVel = paramVals[1]

gDT = paramVals[2]

gDMax = paramVals[3]

gVMax = paramVals[4]

gDTarget = paramVals[5]

gLightLength = paramVals[6]

gDNum = int(paramVals[7])

gVNum = int(paramVals[8])

gNt = int(paramVals[9])

gAlpha = paramVals[10]

gBeta = paramVals[11]

gTG = paramVals[12]

gTR = paramVals[13]

gTerminalT = paramVals[14]
print("Terminal Time: ", gTerminalT)
gAccelInfty = paramVals[15]

gN = gVNum #how it's set in code 

gInfty = 10000000 

forbiddenAccel = np.nan

##========================================STOCHASTIC LIGHT INFO====================================================
if gInUncertainGreenPhase == 1:
    gTY1 = stochLightInfo[0]
    gTY2 = stochLightInfo[1]
    gTYP1 = stochLightInfo[2]
    gTYP2 = stochLightInfo[3]
    gYellowDuration = stochLightInfo[4]
    gRedDuration = stochLightInfo[5]
    gTR = stochLightInfo[6]
    gTG = stochLightInfo[7]

##=======================================SAMPLING DETAILS==========================================================
gSamplingTNumThresh = samplingInfo[0]
gSliceSampleSpacing = samplingInfo[1]

if (gNt < gSamplingTNumThresh):
    gSliceSampleSpacing = 1
       
maxSliceIndex = gNt / gSliceSampleSpacing

print("Maximum Time Index: ", maxSliceIndex)

prohibitedVal = gInfty

Terminal Time:  10000000.0
Maximum Time Index:  40000000.0


## Timeslice Selection:

Enter time index corresponding to the time at which you wish to view the value function contour still.

In [37]:
timeslice = int(input("Timeslice index: ")) #user specified timeslice to plot
sampleSlice = timeslice / gSliceSampleSpacing 
sampleSliceMod = timeslice % gSliceSampleSpacing 
if sampleSlice > maxSliceIndex or sampleSliceMod != 0: 
    print ("Time out of bounds, invalid choice'")
    sys.exit() #kill the program if user-specified N is out of bounds
    
mpl.rcParams['font.size'] = 18

Timeslice index: 0


## Data Processing:

In [38]:
fileName = 'FileName'
fileName2 = 'FileName'
fileName3 = 'FileName'
if problemSelector == 1:
    fileName = '../output/stationary_valuefn'
elif problemSelector == 2:
    fileName = '../output/rg_valuefn'
elif problemSelector == 3:
    fileName = '../output/yrg_valuefn_c2_0.33'
elif problemSelector == 4:
    fileName = '../output/uncertain_green_valuefn_2L_c2_0.33_p1_0.50_p2_0.50'
    fileName2 = '../output/uncertain_green_valuefn_2L_c2_0.33_p1_0.95_p2_0.05'
    fileName3 = '../output/uncertain_green_valuefn_2L_c2_0.75_p1_0.50_p2_0.50'
elif problemSelector == 5:
    fileName = '../output/uncertain_green_valuefn_3L_c2_0.33_p1_0.25_p2_0.25_p3_0.50'

In [39]:
##Select File:
data  = np.loadtxt('%s.txt' %fileName)
data2  = np.loadtxt('%s.txt' %fileName)
data3  = np.loadtxt('%s.txt' %fileName)
if problemSelector == 4:
    data2 = np.loadtxt('%s.txt' %fileName2) 
    data3 = np.loadtxt('%s.txt' %fileName3) 

currentT = timeslice * gDT
print("Current time in process: ", currentT)

#-----------------------------------------Max accel line computation- can be plotted if desired------------------------
timeToRed = gTR - currentT
vCritical = gVMax - gBeta * timeToRed
vThreshLinear = gAlpha * timeToRed * (1 + np.sqrt(1 + (gBeta / gAlpha)))
parabExtensionA = (gBeta + gAlpha) / (2 * gAlpha * gBeta)
parabExtensionB = -gVMax / gBeta
parabExtensionC = (gVMax**2) / (2 * gBeta) -timeToRed * gVMax
parabExtensionVelThresh = (-parabExtensionB + np.sqrt((parabExtensionB**2) - 4 *parabExtensionA * parabExtensionC)) / (2 * parabExtensionA)
vThreshParab = (-parabExtensionB + np.sqrt((parabExtensionB**2) - 4 *parabExtensionA * parabExtensionC)) / (2 * parabExtensionA);
officialVThresh = vThreshLinear
if vCritical < vThreshLinear:
    officialVThresh = vThreshParab 

maxAccelLine = []
velVals = np.linspace(0, gVMax, gVNum)
for i in range (0, len(velVals)):
    v = velVals[i]
    d = 0
    timeToMaxV = (gVMax - v)
    if v < vCritical:
        d = v * (gTR - currentT) + 0.5*gBeta * (gTR - currentT)**2
    else:
        d = (-(v**2) / (2 * gBeta)) + ((v * gVMax) / gBeta) - (0.5 * (gVMax**2) / gBeta) + timeToRed*gVMax  
    maxAccelLine.append(d)
##Starting and ending time indices for matrix extraction 
start = int(timeslice*(gDNum+1)) 
end = int(start + gDNum + 1)

## -----------------------------------------Data processing------------------------------------------------------------
#extract the matrix for the user specifed slice 
plottingData = data[start:end]
plottingData2 = data2[start:end]
plottingData3 = data3[start:end]

#flip the plotting data correctly to plot starting from the bottom left corner
plottingData = np.transpose(plottingData) #rows first become columns 

#Go through data and reset infty to nan
for j in range (0, gVNum+1):
    for i in range (0, gDNum+1):
        if plottingData[j][i] >= gInfty:
            nanVal = np.nan
            plottingData[j][i] = nanVal

if problemSelector == 4:
    data2 = np.loadtxt('%s.txt' %fileName2) 
    data3 = np.loadtxt('%s.txt' %fileName3) 
    start = int(sampleSlice * (gDNum + 1))
    end = int(start + gDNum + 1)

    #extract the matrix for the user specifed slice 
    plottingData2 = data2[start:end]
    plottingData3 = data3[start:end]
    
    #flip the plotting data correctly to plot starting from the bottom left corner
    plottingData2 = np.transpose(plottingData2) #rows first become columns 
    plottingData3 = np.transpose(plottingData3) #rows first become columns 

    #Go through data and reset accel infty to nan
    for j in range (0, gVNum):
        for i in range (0, gDNum):
            if plottingData2[j][i] == prohibitedVal:
                nanVal = np.nan
                plottingData2[j][i] = nanVal
            if plottingData3[j][i] == prohibitedVal:
                nanVal = np.nan
                plottingData3[j][i] = nanVal

Current time in process:  0.0


## Value Function Contour Plotting 

In [41]:
##-------------Meshgrid formation with d,v values------------- 
d = np.linspace(gDTarget, gDMax, gDNum)
v = np.linspace(0, gVMax, gVNum)
d = np.append(d, gDMax)
v = np.append(v, gVMax)
D, V = np.meshgrid(d, v)

##---------------Contour plotting-------------------------
plt.figure(1)

contourPlot = plt.contour(D, V, plottingData, 10, colors = 'black')
plt.clabel(contourPlot, inline=True, fontsize=8) #turn on for contour labels
plt.imshow(plottingData, extent=[gDTarget,gDMax,0,gVMax], origin='lower', alpha=0.75)
#plt.plot(maxAccelLine,velVals, c='white', linewidth = 2, label = '$d_{\beta}$ if $T_{Y} = $')

plt.xlabel('Position ($m$)')
plt.ylabel('Velocity ($m/s$)')
#plt.colorbar(label='$u(d, v, t)$') #Note: change manually to w in uncertain phase
#plt.colorbar(label='$w^2(d, v, t)$') #Note: change manually to w in uncertain phase
plt.colorbar(label='$q(d, v)$') #Note: change manually in indef green
plt.gca().set_aspect('auto')

if problemSelector == 4:
    plt.figure(2)

    contourPlot2 = plt.contour(D, V, plottingData2, 10, colors = 'black')
    plt.clabel(contourPlot2, inline=True, fontsize=8) #turn on for contour labels
    plt.imshow(plottingData2, extent=[gDTarget,gDMax,0,gVMax], origin='lower', alpha=0.75)
    #plt.plot(maxAccelLine,velVals, c='white', linewidth = 2, label = '$d_{\beta}$ if $T_{Y} = $')

    plt.xlabel('Position ($m$)')
    plt.ylabel('Velocity ($m/s$)')
    #plt.colorbar(label='$u(d, v, t)$') 
    plt.colorbar(label='$w^1(d, v, t)$')
    plt.gca().set_aspect('auto')

    plt.figure(3)

    contourPlot3 = plt.contour(D, V, plottingData3, 10, colors = 'black')
    plt.clabel(contourPlot3, inline=True, fontsize=8) #turn on for contour labels
    plt.imshow(plottingData3, extent=[gDTarget,gDMax,0,gVMax], origin='lower', alpha=0.75)
    #plt.plot(maxAccelLine,velVals, c='white', linewidth = 2, label = '$d_{\beta}$ if $T_{Y} = $')

    plt.xlabel('Position ($m$)')
    plt.ylabel('Velocity ($m/s$)')
    #plt.colorbar(label='$u(d, v, t)$')
    plt.colorbar(label='$w^1(d, v, t)$')
    plt.gca().set_aspect('auto')